## Realised that there was a typo in the code and in reality the overlapping columns are very less. Hence the datasets cannot be merged. Donot run this notebook.

# Merge data from studies on 
- Colorectal Adenocarcinoma (TCGA, PanCancer Atlas)
- Colorectal Adenocarcinoma (TCGA, Firehose Legacy)

1. Read data from first study and reproduce steps in colorectal_adenocarcinoma/01_colorectal_preliminary.ipynb to have cleaned data.
1. Read data from second study and try to match it as much as possible to the data from first study.
1. Merge tables.
1. Reproduce results with new formed datasets.

In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from helpers import load_config, plot_predicted_vs_actual
from colorectal_adenocarcinoma.data_preprocessor import preprocess_data, show_unique_vals_by_column
import sys
import os
import warnings
import copy
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from  colorectal_adenocarcinoma.model_evaluators import multiclass_classification, plot_feature_importance_plot_using_rf
import seaborn as sns

warnings.filterwarnings("ignore")


In [3]:
configs = load_config(os.path.join("configs", "configs.yaml"))

## 1. Prepare data from first study

#### 1.1 Reproduce the same steps as notebooks from colorectal_adenocarcinoma/colorectal_multiclass.ipynb

Note: OS_MONTHS instead of PFS_MONTHS is used for target column in this analysis since its available in both studies and seem to be used by many research projects. 

In [13]:
# load patient data
patientdf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY2"], "data_clinical_patient.txt"),
    sep="\t",
    header=4
)

columns_being_discarded =  [
    # 'SUBTYPE', # consider for later analysis
    'CANCER_TYPE_ACRONYM', # same value for all records
    'OTHER_PATIENT_ID', # id column not related to any data of our interest
    'AJCC_STAGING_EDITION', # version of handbook used for classification
    'DAYS_LAST_FOLLOWUP', # similar but more relevant information captured in OS_Months
    'DAYS_TO_BIRTH', # age captures similar information 
    'DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS', # similar but more relevant information captured in OS_Months 
    'ETHNICITY', # race column has more rich data
    'FORM_COMPLETION_DATE', # irrelevant to analysis
    'HISTORY_NEOADJUVANT_TRTYN', # highly skewed, 590 No, 2 Yes, 2 Null
    'ICD_O_3_HISTOLOGY' , # which edition of certain code used for tabulating data
    'ICD_O_3_SITE', # which edition of certain code used for tabulating data
    'INFORMED_CONSENT_VERIFIED', # irrelevant to analysis
    'PRIMARY_LYMPH_NODE_PRESENTATION_ASSESSMENT', # highly skewed 557 Yes, 19 No, 18 Null
    'IN_PANCANPATHWAYS_FREEZE', # if patient is part of a certain analysis, irrelevant
    ]

patientdf_1 = patientdf.drop(columns=columns_being_discarded)

# load sample data
sampledf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY2"], "data_clinical_sample.txt"),
    sep="\t",
    header=4
)

columns_being_discarded_sampledf = [
    'TISSUE_PROSPECTIVE_COLLECTION_INDICATOR', # when sample was collected, before or after
    'TISSUE_RETROSPECTIVE_COLLECTION_INDICATOR', # when sample was collected, before or after
    'TISSUE_SOURCE_SITE_CODE', # which hospital was the sample collected at
    'TUMOR_TISSUE_SITE', # part of body where sample was collected
    'SAMPLE_TYPE', # has only one value "Primary"
    'TISSUE_SOURCE_SITE' # which hospital was the sample collected at
]

sampledf_1 = sampledf.drop(columns=columns_being_discarded_sampledf)

# merge patient and sample 
merge_df = pd.merge(patientdf_1, sampledf_1, on='PATIENT_ID', how='inner')

# drop columns based on analysis done in the reference notebook
columns_to_drop = [
    'GRADE',
    'DFS_MONTHS', 'DFS_STATUS',
    'SUBTYPE', 'ICD_10',
    'NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT',
    'DSS_STATUS', 'DSS_MONTHS',
    # 'AJCC_PATHOLOGIC_TUMOR_STAGE', not removing for now since new dataset has many values for column
    'SOMATIC_STATUS',
    'TUMOR_TYPE',
    'CANCER_TYPE',
    'RACE',
    'CANCER_TYPE_DETAILED'
]

df = merge_df.drop(columns=columns_to_drop)

# drop any row with missing value
df1 = df.dropna()

# keep only cases with recurrence
df1 = df1[df1["PFS_STATUS"] == "1:PROGRESSION"]
df1.drop(["PFS_STATUS"], axis=1, inplace=True)

# filter out rows with missing value for race since we plan to merge data with other dataset, so trying to retain high-quality data only.
df1 = df1[df1.GENETIC_ANCESTRY_LABEL != " "]

# save results for merging later
df1.to_csv(
    os.path.join(
        configs["STORAGE_DIR_STUDY2"], "recurrence_clinical.csv"),
    index=False
    )

study1_df1 = df1.copy()

Steps before passing into the model
1. pass it thru preprocess_data function.
2. separate out y.

##### Data Specifications for the pipeline:
1. GENETIC_ANCESTRY_LABEL: 
    - has values like [' ', 'AFR', 'AFR_ADMIX', 'EAS', 'EUR', 'EUR_ADMIX']
    - highly skewed toward EUR. 
    - output 1 for EUR(white), 0 for non-EUR(non-white).
    - potentially modify later to include other races if good number of records are available.

1. PRIOR_DX
    - starts with either Yes or No. 

1. PATH_M_STAGE
    - contains values out of [M0, M1, M1A, M1B, MX]

1. PATH_T_STAGE
    - contains values out of ['T1', 'T2', 'T3', 'T4', 'T4A', 'T4B', 'TIS']

1. PATH_N_STAGE
    - contains values out of ['N0', 'N1', 'N1A', 'N1B', 'N1C', 'N2', 'N2A', 'N2B'] 

1. All other columns are either object type or int64 type.




In [5]:
print("Column in Study 2:")
for column in sorted(df1.columns):
    print(column)


Column in Study 2:
AGE
AJCC_PATHOLOGIC_TUMOR_STAGE
ANEUPLOIDY_SCORE
GENETIC_ANCESTRY_LABEL
MSI_SCORE_MANTIS
MSI_SENSOR_SCORE
ONCOTREE_CODE
PATH_M_STAGE
PATH_N_STAGE
PATH_T_STAGE
PATIENT_ID
PERSON_NEOPLASM_CANCER_STATUS
PFS_MONTHS
PRIOR_DX
RADIATION_THERAPY
SAMPLE_ID
SEX
TMB_NONSYNONYMOUS
WEIGHT


Used study2/user_friendly_formats/recurrence_clinical_edited.xlsx for matching column name to column definitions

In [6]:
pd.read_excel(
    os.path.join(configs["STORAGE_DIR_STUDY2"], "user_friendly_formats", "recurrence_clinical_edited.xlsx"),
    sheet_name="Used_Column_Defintions"
)

,PATIENT_ID,#Patient Identifier
0,AGE,Diagnosis Age
1,SEX,Sex
2,PATH_M_STAGE,American Joint Committee on Cancer Metastasis ...
3,PATH_N_STAGE,Neoplasm Disease Lymph Node Stage American Joi...
4,PATH_T_STAGE,American Joint Committee on Cancer Tumor Stage...
5,PERSON_NEOPLASM_CANCER_STATUS,Person Neoplasm Cancer Status
6,PRIOR_DX,Prior Diagnosis
7,RADIATION_THERAPY,Radiation Therapy
8,WEIGHT,Patient Weight
9,PFS_MONTHS,Progress Free Survival (Months)


## 2. Clean data from second study 

#### 2.1 Join Patient and Sample data and analyze

In [42]:
patientdf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "data_clinical_patient.txt"),
    sep="\t"
    # header=4
)
patientdf.head()

patientdf.to_excel(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "user_friendly_formats", "patientdf.xlsx"),
    index=False)

In [43]:
# load sample data
sampledf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "data_clinical_sample.txt"),
    sep="\t"
    # header=4
)

sampledf.to_excel(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "user_friendly_formats", "sampledf.xlsx"),
    index=False)

Analyzed data in excel using study3/user_friendly_formats/patientdf_edited and enlisted columns that are not needed since they donot have a match in study 1.

In [38]:
patientdf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "data_clinical_patient.txt"),
    sep="\t",
    header=4
)
sampledf = pd.read_csv(
    os.path.join(configs["STORAGE_DIR_STUDY3"], "data_clinical_sample.txt"),
    sep="\t",
    header=4
)

merge_df = pd.merge(patientdf, sampledf, on='PATIENT_ID', how='inner')
merge_df.shape

(633, 86)

#### 2.2 Remove useless columns 

In [39]:
columns_to_drop = [
    "OTHER_PATIENT_ID",
    "FORM_COMPLETION_DATE",
    "HISTOLOGICAL_DIAGNOSIS",
    "PROSPECTIVE_COLLECTION",
    "RETROSPECTIVE_COLLECTION",
    "ETHNICITY",
    "HISTORY_NEOADJUVANT_TRTYN",
    "INITIAL_PATHOLOGIC_DX_YEAR",
    "AJCC_STAGING_EDITION",
    "RESIDUAL_TUMOR",
    "LYMPH_NODES_EXAMINED",
    "LYMPH_NODE_EXAMINED_COUNT",
    "LYMPH_NODES_EXAMINED_HE_COUNT",
    "LYMPH_NODES_EXAMINED_IHC_COUNT",
    "VASCULAR_INVASION_INDICATOR",
    "LYMPHOVASCULAR_INVASION_INDICATOR",
    "PERINEURAL_INVASION",
    "KRAS_GENE_ANALYSIS_INDICATOR",
    "KRAS_MUTATION",
    "BRAF_GENE_ANALYSIS_INDICATOR",
    "BRAF_GENE_ANALYSIS_RESULT",
    "HEIGHT",
    "RADIATION_TREATMENT_ADJUVANT",
    "PHARMACEUTICAL_TX_ADJUVANT",
    "TREATMENT_OUTCOME_FIRST_COURSE",
    "NEW_TUMOR_EVENT_AFTER_INITIAL_TREATMENT",
    "PRIMARY_SITE_PATIENT",
    "CLIN_M_STAGE",
    "CLIN_N_STAGE",
    "CLIN_T_STAGE",
    "CLINICAL_STAGE",
    "DAYS_TO_INITIAL_PATHOLOGIC_DIAGNOSIS",
    "DAYS_TO_PATIENT_PROGRESSION_FREE",
    "DAYS_TO_TUMOR_PROGRESSION",
    "DISEASE_CODE",
    "EXTRANODAL_INVOLVEMENT",
    "ICD_10",
    "ICD_O_3_HISTOLOGY",
    "ICD_O_3_SITE",
    "INFORMED_CONSENT_VERIFIED",
    "INITIAL_PATHOLOGIC_DIAGNOSIS_METHOD",
    "PROJECT_CODE",
    "STAGE_OTHER",
    "TISSUE_SOURCE_SITE_x",
    "TISSUE_SOURCE_SITE_y",
    "SITE_OF_TUMOR_TISSUE",
    "CANCER_TYPE",
    "CANCER_TYPE_DETAILED",
    "TUMOR_TYPE",
    "GRADE",
    "TISSUE_PROSPECTIVE_COLLECTION_INDICATOR",
    "TISSUE_RETROSPECTIVE_COLLECTION_INDICATOR",
    "TISSUE_SOURCE_SITE_CODE",
    "TUMOR_TISSUE_SITE",
    "SAMPLE_TYPE",
    "SOMATIC_STATUS",
    "OS_MONTHS", 
    "OS_STATUS"
]

In [40]:
df = merge_df.drop(columns=columns_to_drop, axis=1)

KeyError: "['TISSUE_SOURCE_SITE_x', 'TISSUE_SOURCE_SITE_y', 'TUMOR_TYPE', 'GRADE', 'TISSUE_PROSPECTIVE_COLLECTION_INDICATOR', 'TISSUE_RETROSPECTIVE_COLLECTION_INDICATOR', 'TISSUE_SOURCE_SITE_CODE', 'TUMOR_TISSUE_SITE'] not found in axis"

#### 2.3 Match columns in both tables and join

In [28]:
df = df.rename(columns=
               {
                    "AJCC_METASTASIS_PATHOLOGIC_PM": "PATH_M_STAGE",
                    "AJCC_NODES_PATHOLOGIC_PN": "PATH_N_STAGE",
                    "AJCC_TUMOR_PATHOLOGIC_PT": "PATH_T_STAGE",
                    "DFS_MONTHS": "PFS_MONTHS",
                    "DFS_STATUS": "PFS_STATUS",
                    "HISTORY_OTHER_MALIGNANCY": "PRIOR_DX"
                })

In [29]:
# datatype corrections:
df["PFS_MONTHS"].replace('[Not Available]', None, inplace=True)
df['PFS_MONTHS'] = pd.to_numeric(df['PFS_MONTHS'])

In [14]:
df["WEIGHT"].replace('[Not Available]', None, inplace=True)

df['WEIGHT'] = pd.to_numeric(df['WEIGHT'])

In [30]:
df1 = df[df["PFS_STATUS"] == "1:Recurred/Progressed"]
df1 = df1.drop(["PFS_STATUS"], axis=1)
df.shape, df1.shape

((591, 19), (111, 18))

In [16]:
study2_df1 = df1.copy()

In [17]:
show_unique_vals_by_column(study2_df1)

Column 'AJCC_PATHOLOGIC_TUMOR_STAGE' has 12 unique values: ['Stage I', 'Stage II', 'Stage IIA', 'Stage IIB', 'Stage III', 'Stage IIIA', 'Stage IIIB', 'Stage IIIC', 'Stage IV', 'Stage IVA'] ...
Column 'ONCOTREE_CODE' has 3 unique values: ['COAD', 'MACR', 'READ'] 
Column 'PATH_M_STAGE' has 6 unique values: ['M0', 'M1', 'M1a', 'M1b', 'MX', '[Not Available]'] 
Column 'PATH_N_STAGE' has 7 unique values: ['N0', 'N1', 'N1a', 'N1b', 'N2', 'N2a', 'N2b'] 
Column 'PATH_T_STAGE' has 6 unique values: ['T1', 'T2', 'T3', 'T4', 'T4a', 'T4b'] 
Column 'PATIENT_ID' has 122 unique values: ['TCGA-5M-AAT4', 'TCGA-5M-AAT6', 'TCGA-A6-2671', 'TCGA-A6-2676', 'TCGA-A6-2677', 'TCGA-A6-2682', 'TCGA-A6-2683', 'TCGA-A6-2686', 'TCGA-A6-4105', 'TCGA-A6-A565'] ...
Column 'PRIOR_DX' has 2 unique values: ['No', 'Yes'] 
Column 'RACE' has 4 unique values: ['ASIAN', 'BLACK OR AFRICAN AMERICAN', 'WHITE', '[Not Available]'] 
Column 'SAMPLE_ID' has 122 unique values: ['TCGA-5M-AAT4-01', 'TCGA-5M-AAT6-01', 'TCGA-A6-2671-01', 'T

Columns with missing values: 
1. RACE: [Not Available]
2. TUMOR_STATUS: '[Discrepancy]', '[Not Available]'

In [18]:
study1_df1.RADIATION_THERAPY.value_counts()

RADIATION_THERAPY
No     56
Yes     4
Name: count, dtype: int64

RADIATION_THERAPY has very skewed values and is not present in second study, so removing it.

In [19]:
study1_df1 = study1_df1.drop("RADIATION_THERAPY", axis=1)

study1_df1 = study1_df1.rename(columns={
    "PERSON_NEOPLASM_CANCER_STATUS": "TUMOR_STATUS",
    "GENETIC_ANCESTRY_LABEL": "RACE"
})

In [20]:
show_unique_vals_by_column(study1_df1)

Column 'AJCC_PATHOLOGIC_TUMOR_STAGE' has 9 unique values: ['STAGE I', 'STAGE IIA', 'STAGE IIB', 'STAGE IIC', 'STAGE IIIB', 'STAGE IIIC', 'STAGE IV', 'STAGE IVA', 'STAGE IVB'] 
Column 'ONCOTREE_CODE' has 3 unique values: ['COAD', 'MACR', 'READ'] 
Column 'PATH_M_STAGE' has 5 unique values: ['M0', 'M1', 'M1A', 'M1B', 'MX'] 
Column 'PATH_N_STAGE' has 8 unique values: ['N0', 'N1', 'N1A', 'N1B', 'N1C', 'N2', 'N2A', 'N2B'] 
Column 'PATH_T_STAGE' has 5 unique values: ['T2', 'T3', 'T4', 'T4A', 'T4B'] 
Column 'PATIENT_ID' has 60 unique values: ['TCGA-5M-AAT6', 'TCGA-A6-2671', 'TCGA-A6-2674', 'TCGA-A6-2681', 'TCGA-A6-2683', 'TCGA-A6-2684', 'TCGA-A6-2685', 'TCGA-A6-4105', 'TCGA-A6-5664', 'TCGA-A6-5665'] ...
Column 'PRIOR_DX' has 2 unique values: ['No', 'Yes'] 
Column 'RACE' has 4 unique values: ['AFR', 'AFR_ADMIX', 'EAS', 'EUR'] 
Column 'SAMPLE_ID' has 60 unique values: ['TCGA-5M-AAT6-01', 'TCGA-A6-2671-01', 'TCGA-A6-2674-01', 'TCGA-A6-2681-01', 'TCGA-A6-2683-01', 'TCGA-A6-2684-01', 'TCGA-A6-2685-

In [21]:
# ensuring both studies have same columns
for column in study1_df1.columns:
    if column not in study2_df1:
        print(column)

for column in study2_df1.columns:
    if column not in study1_df1:
        print(column)


In [22]:
study1_df1["STUDY_NAME"] = "STUDY1"
study2_df1["STUDY_NAME"] = "STUDY2"


In [23]:
df = pd.concat([study1_df1, study2_df1], axis=0)
df.shape

(182, 19)

In [24]:
has_duplicates = df['PATIENT_ID'].duplicated().any()
print(f"Column 'PATIENT_ID' has duplicates: {has_duplicates}")

df["PATIENT_ID"].value_counts().sort_values(ascending=False)

repeating_values_rows = df[df['PATIENT_ID'].duplicated(keep=False)]

# Display the rows
print("Rows with repeating values in column 'PATIENT_ID':")
repeating_values_rows.sort_values(by="PATIENT_ID")



Column 'PATIENT_ID' has duplicates: True
Rows with repeating values in column 'PATIENT_ID':


,PATIENT_ID,AGE,SEX,AJCC_PATHOLOGIC_TUMOR_STAGE,PATH_M_STAGE,PATH_N_STAGE,PATH_T_STAGE,TUMOR_STATUS,PRIOR_DX,WEIGHT,PFS_MONTHS,RACE,SAMPLE_ID,ONCOTREE_CODE,ANEUPLOIDY_SCORE,MSI_SCORE_MANTIS,MSI_SENSOR_SCORE,TMB_NONSYNONYMOUS,STUDY_NAME
4,TCGA-5M-AAT6,40.0,Female,STAGE IV,M1A,N2B,T4A,With Tumor,No,99.1,7.199921,AFR,TCGA-5M-AAT6-01,COAD,3.0,0.6600,20.58,62.166667,STUDY1
5,TCGA-5M-AAT6,40.0,Female,Stage IV,M1a,N2b,T4a,WITH TUMOR,No,99.1,9.530000,BLACK OR AFRICAN AMERICAN,TCGA-5M-AAT6-01,COAD,3.0,0.6600,20.58,62.166667,STUDY2
7,TCGA-A6-2671,85.0,Male,Stage IV,M1,N2,T3,WITH TUMOR,No,67.2,43.730000,WHITE,TCGA-A6-2671-01,COAD,25.0,0.3500,0.37,4.066667,STUDY2
6,TCGA-A6-2671,85.0,Male,STAGE IV,M1,N2,T3,With Tumor,No,67.2,17.588848,EUR,TCGA-A6-2671-01,COAD,25.0,0.3500,0.37,4.066667,STUDY1
18,TCGA-A6-2683,57.0,Female,Stage IV,M1,N0,T4,WITH TUMOR,No,57.5,16.560000,WHITE,TCGA-A6-2683-01,COAD,20.0,0.3644,1.46,2.133333,STUDY2
17,TCGA-A6-2683,57.0,Female,STAGE IV,M1,N0,T4,With Tumor,No,57.5,9.862906,EUR,TCGA-A6-2683-01,COAD,20.0,0.3644,1.46,2.133333,STUDY1
26,TCGA-A6-4105,79.0,Male,Stage IIA,M0,N0,T3,TUMOR FREE,Yes,83.3,14.520000,WHITE,TCGA-A6-4105-01,COAD,17.0,0.3528,0.30,4.866667,STUDY2
25,TCGA-A6-4105,79.0,Male,STAGE IIA,M0,N0,T3,Tumor Free,Yes,83.3,11.966992,EUR,TCGA-A6-4105-01,COAD,17.0,0.3528,0.30,4.866667,STUDY1
53,TCGA-A6-A565,34.0,Female,Stage IIIC,MX,N2,T3,WITH TUMOR,No,66.7,16.230000,BLACK OR AFRICAN AMERICAN,TCGA-A6-A565-01,MACR,0.0,0.3972,0.41,12.200000,STUDY2
52,TCGA-A6-A565,34.0,Female,STAGE IIIC,MX,N2,T3,With Tumor,No,66.7,9.895782,AFR,TCGA-A6-A565-01,MACR,0.0,0.3972,0.41,12.200000,STUDY1


## 3. Prepare X using merged table

#### 3.1 Find columns needing more cleanup

In [25]:
show_unique_vals_by_column(df.drop(["PATIENT_ID", "SAMPLE_ID"], axis=1))

Column 'AJCC_PATHOLOGIC_TUMOR_STAGE' has 21 unique values: ['STAGE I', 'STAGE IIA', 'STAGE IIB', 'STAGE IIC', 'STAGE IIIB', 'STAGE IIIC', 'STAGE IV', 'STAGE IVA', 'STAGE IVB', 'Stage I'] ...
Column 'ONCOTREE_CODE' has 3 unique values: ['COAD', 'MACR', 'READ'] 
Column 'PATH_M_STAGE' has 8 unique values: ['M0', 'M1', 'M1A', 'M1B', 'M1a', 'M1b', 'MX', '[Not Available]'] 
Column 'PATH_N_STAGE' has 12 unique values: ['N0', 'N1', 'N1A', 'N1B', 'N1C', 'N1a', 'N1b', 'N2', 'N2A', 'N2B'] ...
Column 'PATH_T_STAGE' has 8 unique values: ['T1', 'T2', 'T3', 'T4', 'T4A', 'T4B', 'T4a', 'T4b'] 
Column 'PRIOR_DX' has 2 unique values: ['No', 'Yes'] 
Column 'RACE' has 8 unique values: ['AFR', 'AFR_ADMIX', 'ASIAN', 'BLACK OR AFRICAN AMERICAN', 'EAS', 'EUR', 'WHITE', '[Not Available]'] 
Column 'SEX' has 2 unique values: ['Female', 'Male'] 
Column 'STUDY_NAME' has 2 unique values: ['STUDY1', 'STUDY2'] 
Column 'TUMOR_STATUS' has 6 unique values: ['TUMOR FREE', 'Tumor Free', 'WITH TUMOR', 'With Tumor', '[Discre

Columns needing more cleaning:
- AJCC_PATHOLOGIC_TUMOR_STAGE
- PATH_M_STAGE
- RACE
- TUMOR_STATUS

#### 3.2 Clean Tumor Stage

In [26]:
df["AJCC_PATHOLOGIC_TUMOR_STAGE"].value_counts()

AJCC_PATHOLOGIC_TUMOR_STAGE
Stage IV           31
Stage IIA          22
Stage IIIC         20
STAGE IIA          13
Stage IIIB         13
STAGE IV           11
STAGE IIIC         10
STAGE IIIB         10
STAGE IVA           9
Stage II            8
Stage I             7
Stage IVA           7
[Not Available]     5
Stage III           4
STAGE I             3
[Discrepancy]       2
Stage IIB           2
STAGE IIB           2
STAGE IVB           1
STAGE IIC           1
Stage IIIA          1
Name: count, dtype: int64

In [27]:
df["AJCC_PATHOLOGIC_TUMOR_STAGE"] = df["AJCC_PATHOLOGIC_TUMOR_STAGE"].replace(
    '[Not Available]', None
).replace(
    '[Discrepancy]', None
    )

df["AJCC_PATHOLOGIC_TUMOR_STAGE"].value_counts(dropna=False)


AJCC_PATHOLOGIC_TUMOR_STAGE
Stage IV      31
Stage IIA     22
Stage IIIC    20
STAGE IIA     13
Stage IIIB    13
STAGE IV      11
STAGE IIIC    10
STAGE IIIB    10
STAGE IVA      9
Stage II       8
Stage IVA      7
None           7
Stage I        7
Stage III      4
STAGE I        3
Stage IIB      2
STAGE IIB      2
STAGE IVB      1
STAGE IIC      1
Stage IIIA     1
Name: count, dtype: int64

Note: T Stage and Tumor Stage might be same, check correlation later.

#### 3.2 Path M Stage

In [28]:
df["PATH_M_STAGE"].value_counts()

PATH_M_STAGE
M0                 95
M1                 47
MX                 26
M1A                 6
M1a                 3
[Not Available]     3
M1B                 1
M1b                 1
Name: count, dtype: int64

In [29]:
df["PATH_M_STAGE"] = df["PATH_M_STAGE"].replace(
    '[Not Available]', "MX"
)
df["PATH_M_STAGE"].value_counts()

PATH_M_STAGE
M0     95
M1     47
MX     29
M1A     6
M1a     3
M1B     1
M1b     1
Name: count, dtype: int64

#### 3.3 Race

In [30]:
df["RACE"].value_counts()

RACE
WHITE                        69
EUR                          40
[Not Available]              38
AFR                          15
BLACK OR AFRICAN AMERICAN    13
AFR_ADMIX                     3
EAS                           2
ASIAN                         2
Name: count, dtype: int64

In [31]:
print(f"White: {69+40}")
print(f"African: {15+13+3}")

White: 109
African: 31
